<a href="https://colab.research.google.com/github/rorothegreat/430L_Project/blob/main/Traffic_Accident_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import os

# Importing the Dataset and Cleaning the Data

In [ ]:
import pandas as pd
import requests
import zipfile
import io

# Download and load the traffic accident dataset
!wget https://static.nhtsa.gov/nhtsa/downloads/FARS/2020/National/FARS2020NationalCSV.zip
!unzip FARS2020NationalCSV.zip


In [ ]:
import pandas as pd

# Load the 'accident.csv' file into a Pandas DataFrame
accident_df = pd.read_csv('/content/FARS2020NationalCSV/accident.csv', encoding='ISO-8859-1')

# Select only relevant columns
keep_cols = ['ST_CASE', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PEDS', 'PERSONS', 'PERMVIT', 'PERNOTMVIT', 'DAY_WEEK', 'HOUR', 'MINUTE', 'NHS', 'ROUTE', 'RUR_URB', 'FUNC_SYS', 'RD_OWNER', 'ROUTE', 'SP_JUR', 'MAN_COLL', 'RELJCT1', 'RELJCT2', 'TYP_INT', 'WRK_ZONE', 'WEATHER', 'DRUNK_DR']
accident_df = accident_df[keep_cols]

# Load the 'person.csv' file into a Pandas DataFrame
person_df = pd.read_csv('/content/FARS2020NationalCSV/person.csv', encoding='ISO-8859-1')

# Select only relevant columns
keep_cols = ['ST_CASE', 'VEH_NO', 'PER_NO', 'PER_TYP', 'AGE', 'SEX', 'INJ_SEV', 'REST_USE', 'AIR_BAG', 'EJECTION', 'DRINKING']
person_df = person_df[keep_cols]

# Merge the 'person_df' DataFrame with the 'accident_df' DataFrame
accident_df = pd.merge(accident_df, person_df, on='ST_CASE', how='inner')

# Load the 'vehicle.csv' file into a Pandas DataFrame
vehicle_df = pd.read_csv('/content/FARS2020NationalCSV/vehicle.csv', encoding='ISO-8859-1')

# Select only relevant columns
keep_cols = ['STATE', 'ST_CASE', 'VEH_NO', 'MAKE', 'BODY_TYP', 'MOD_YEAR', 'TRAV_SP', 'ROLLOVER']
vehicle_df = vehicle_df[keep_cols]

# Merge the 'vehicle_df' DataFrame with the 'accident_df' DataFrame
accident_df = pd.merge(accident_df, vehicle_df, on=['ST_CASE', 'VEH_NO'], how='inner')

# Save the merged dataset
accident_df.to_csv('accidents_merged.csv', index=False)


In [ ]:

# Load the merged dataset
merged_df = pd.read_csv('/content/accidents_merged.csv')

# Calculate the percentage of accidents for each hour
hour_counts = merged_df['HOUR'].value_counts(normalize=True) * 100

# Display the results
print(hour_counts)

In [20]:

# Load the merged accident data
accidents_df = pd.read_csv('accidents_merged.csv')

# Convert categorical columns to categorical data type
cat_cols = ['STATE', 'MAKE', 'BODY_TYP', 'WEATHER', 'DRUNK_DR']
accidents_df[cat_cols] = accidents_df[cat_cols].astype('category')

# Remove incorrect hour values
accidents_df['HOUR'] = pd.to_datetime(accidents_df['HOUR'].astype(str).str.zfill(2) + ':00', format='%H:%M', errors='coerce')
accidents_df.loc[accidents_df['HOUR'].dt.hour == 99, 'HOUR'] = np.nan

# Convert time columns to datetime data type
accidents_df['HOUR'] = pd.to_datetime(accidents_df['HOUR'].astype(str).str.zfill(2) + ':00', format='%H:%M').dt.time
accidents_df['MINUTE'] = pd.to_datetime(accidents_df['MINUTE'].astype(str).str.zfill(2), format='%M').dt.time

# Replace missing values in 'TRAV_SP' column with median
accidents_df['TRAV_SP'].fillna(accidents_df['TRAV_SP'].median(), inplace=True)

# Remove outliers in 'TRAV_SP' column
accidents_df = accidents_df[(accidents_df['TRAV_SP'] >= 0) & (accidents_df['TRAV_SP'] <= 120)]

# Remove rows with missing values in relevant columns
accidents_df.dropna(subset=['MAKE', 'BODY_TYP', 'MOD_YEAR', 'HOUR'], inplace=True)

# Remove duplicates
accidents_df.drop_duplicates(inplace=True)

# Save cleaned data to new file
accidents_df.to_csv('accidents_cleaned.csv', index=False)


ValueError: ignored